In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# === LEITURA E AJUSTES DO CSV ===
df = pd.read_csv("dados_rios_ficticios.csv", encoding="utf-8", sep=";")

# Ver e depurar nomes reais (opcional)
print("Colunas detectadas:", df.columns.tolist())

# Limpar BOM, espaços e padronizar nomes
df.columns = [col.encode('utf-8').decode('utf-8-sig').strip().lower() for col in df.columns]

# Renomear automaticamente a coluna de data
for col in df.columns:
    if "data" in col:
        df.rename(columns={col: "data"}, inplace=True)
        break

# Agora deve funcionar sem erro
df["data"] = pd.to_datetime(df["data"], format="%d/%m/%Y", errors="coerce")


# === CATEGORIZAÇÃO DO NÍVEL DO RIO ===
def classificar_nivel(nivel):
    if nivel < 5.5:
        return "Normal"
    elif 5.5 <= nivel < 7.5:
        return "Atenção"
    else:
        return "Risco"

df["nivel_categoria"] = df["nivel_metro"].apply(classificar_nivel)

# === ENGENHARIA DE FEATURES ===
df["dia"] = df["data"].dt.day
df["mes"] = df["data"].dt.month

# Codificação do nome do rio
le = LabelEncoder()
df["rio_codificado"] = le.fit_transform(df["rio"])

# === TREINAMENTO DO MODELO ===
X = df[["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"]]
y = df["nivel_categoria"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# === INTERFACE COM O USUÁRIO FINAL ===
mapa_rios = dict(zip(le.classes_, le.transform(le.classes_)))

print("🔍 Previsão de Risco do Nível do Rio")
print("Rios disponíveis:", ", ".join(mapa_rios.keys()))

rio = input("Digite o nome do rio: ")
while rio not in mapa_rios:
    rio = input("Nome inválido. Digite novamente: ")

chuva = float(input("Digite a quantidade de chuva (mm): "))
vazao = float(input("Digite a vazão do rio (m³/s): "))
dia = int(input("Digite o dia do mês (1-31): "))
mes = int(input("Digite o mês (1-12): "))

# Montar entrada para o modelo
entrada = pd.DataFrame([[chuva, vazao, dia, mes, mapa_rios[rio]]],
                       columns=["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"])

# Prever o risco
risco = modelo.predict(entrada)[0]
print(f"\n📢 Resultado: O nível do rio está em estado de **{risco.upper()}**.")


Colunas detectadas: ['data', 'rio', 'nivel_metro', 'chuva_mm', 'vazao_m3s']
🔍 Previsão de Risco do Nível do Rio
Rios disponíveis: Rio Amazonas, Rio Negro, Rio Solimões, Rio São Francisco, Rio Tocantins

📢 Resultado: O nível do rio está em estado de **RISCO**.


In [35]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# === LEITURA E AJUSTES DO CSV ===
df = pd.read_csv("dados_rios_ficticios.csv", encoding="utf-8", sep=";")

# Limpar BOM, espaços e padronizar nomes
df.columns = [col.encode('utf-8').decode('utf-8-sig').strip().lower() for col in df.columns]

# Renomear automaticamente a coluna de data
for col in df.columns:
    if "data" in col:
        df.rename(columns={col: "data"}, inplace=True)
        break

# Converter a coluna de data
df["data"] = pd.to_datetime(df["data"], format="%d/%m/%Y", errors="coerce")

# === CATEGORIZAÇÃO DO NÍVEL DO RIO ===
def classificar_nivel(nivel):
    if nivel < 5.5:
        return "Normal"
    elif 5.5 <= nivel < 7.5:
        return "Atenção"
    else:
        return "Risco"

df["nivel_categoria"] = df["nivel_metro"].apply(classificar_nivel)

# === ENGENHARIA DE FEATURES ===
df["dia"] = df["data"].dt.day
df["mes"] = df["data"].dt.month

# Codificação do nome do rio
le = LabelEncoder()
df["rio_codificado"] = le.fit_transform(df["rio"])

# === TREINAMENTO DO MODELO ===
X = df[["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"]]
y = df["nivel_categoria"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# === INTERFACE COM O USUÁRIO FINAL (ACESSÍVEL) ===
mapa_rios = dict(zip(le.classes_, le.transform(le.classes_)))

print("🔍 Previsão de Risco do Nível do Rio")
print("Rios disponíveis:", ", ".join(mapa_rios.keys()))

# Entrada do nome do rio
rio = input("Digite o nome do rio: ")
while rio not in mapa_rios:
    rio = input("Nome inválido. Digite novamente: ")

# Entrada da intensidade da chuva (versão acessível)
print("\n🌧️ Sobre a chuva recente na sua região:")
print("1. Não choveu")
print("2. Chuva fraca")
print("3. Chuva média")
print("4. Chuva forte")

chuva_opcao = input("Selecione o nível de chuva (1 a 4): ")

chuva_map = {
    "1": 0.0,
    "2": 5.0,
    "3": 20.0,
    "4": 50.0
}

while chuva_opcao not in chuva_map:
    chuva_opcao = input("Entrada inválida. Escolha de 1 a 4: ")

chuva = chuva_map[chuva_opcao]

# Entrada da data
data_input = input("Digite a data de hoje ou da observação (formato DD/MM/AAAA): ")
data_formatada = pd.to_datetime(data_input, format="%d/%m/%Y", errors="coerce")

if pd.isna(data_formatada):
    print("❌ Data inválida. Tente novamente.")
    exit()

dia = data_formatada.day
mes = data_formatada.month

# Cálculo da vazão média com base no histórico
df_filtrado = df[(df["rio"] == rio) & (df["mes"] == mes)]
if df_filtrado.empty:
    print("❌ Sem dados históricos para esse rio nesse mês.")
    exit()

vazao_media = df_filtrado["vazao_m3s"].mean()

# Montar entrada para o modelo
entrada = pd.DataFrame([[chuva, vazao_media, dia, mes, mapa_rios[rio]]],
                       columns=["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"])

# Prever o risco
risco = modelo.predict(entrada)[0]
print(f"\n📢 Resultado: O nível do rio está em estado de **{risco.upper()}**.")


🔍 Previsão de Risco do Nível do Rio
Rios disponíveis: Rio Amazonas, Rio Negro, Rio Solimões, Rio São Francisco, Rio Tocantins

🌧️ Sobre a chuva recente na sua região:
1. Não choveu
2. Chuva fraca
3. Chuva média
4. Chuva forte

📢 Resultado: O nível do rio está em estado de **RISCO**.


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# === LEITURA E AJUSTES DO CSV ===
df = pd.read_csv("dados_rios_ficticios.csv", encoding="utf-8", sep=";")
df.columns = [col.encode('utf-8').decode('utf-8-sig').strip().lower() for col in df.columns]

# Renomear automaticamente a coluna de data
for col in df.columns:
    if "data" in col:
        df.rename(columns={col: "data"}, inplace=True)
        break

df["data"] = pd.to_datetime(df["data"], format="%d/%m/%Y", errors="coerce")

# === CATEGORIZAÇÃO DO NÍVEL DO RIO ===
def classificar_nivel(nivel):
    if nivel < 5.5:
        return "Normal"
    elif 5.5 <= nivel < 7.5:
        return "Atenção"
    else:
        return "Risco"

df["nivel_categoria"] = df["nivel_metro"].apply(classificar_nivel)
df["dia"] = df["data"].dt.day
df["mes"] = df["data"].dt.month

# Codificação do nome do rio
le = LabelEncoder()
df["rio_codificado"] = le.fit_transform(df["rio"])

# === TREINAMENTO DO MODELO ===
X = df[["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"]]
y = df["nivel_categoria"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# === INTERFACE SIMPLES COM O USUÁRIO ===
mapa_rios = dict(zip(le.classes_, le.transform(le.classes_)))

print("🔍 Previsão de Risco do Nível do Rio")
print("Rios disponíveis:", ", ".join(mapa_rios.keys()))

# Nome do rio
rio = input("Digite o nome do rio: ")
while rio not in mapa_rios:
    rio = input("Nome inválido. Digite novamente: ")

# Entrada da chuva (simples)
print("\n🌧️ Sobre a chuva recente na sua região:")
print("1. Não choveu")
print("2. Chuva fraca")
print("3. Chuva média")
print("4. Chuva forte")

chuva_opcao = input("Selecione o nível de chuva (1 a 4): ")
chuva_map = {"1": 0.0, "2": 5.0, "3": 20.0, "4": 50.0}
while chuva_opcao not in chuva_map:
    chuva_opcao = input("Entrada inválida. Escolha de 1 a 4: ")

chuva = chuva_map[chuva_opcao]

# Data
data_input = input("Digite a data da observação (formato DD/MM/AAAA): ")
data_formatada = pd.to_datetime(data_input, format="%d/%m/%Y", errors="coerce")
if pd.isna(data_formatada):
    print("❌ Data inválida. Tente novamente.")
    exit()

dia = data_formatada.day
mes = data_formatada.month

# Vazão histórica média
df_filtrado = df[(df["rio"] == rio) & (df["mes"] == mes)]
if df_filtrado.empty:
    print("❌ Sem dados históricos para esse rio nesse mês.")
    exit()

vazao_media = df_filtrado["vazao_m3s"].mean()

# Entrada para o modelo
entrada = pd.DataFrame([[chuva, vazao_media, dia, mes, mapa_rios[rio]]],
                       columns=["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"])

# Previsão
risco = modelo.predict(entrada)[0]
print(f"\n📢 Resultado: O nível do rio está em estado de **{risco.upper()}**.")

# === GERAÇÃO DE GRÁFICOS PARA O FRONT ===

# 1. Distribuição de categorias de risco
plt.figure(figsize=(6, 4))
df["nivel_categoria"].value_counts().plot(kind="bar")
plt.title("Distribuição de Risco do Nível do Rio")
plt.xlabel("Categoria")
plt.ylabel("Número de Ocorrências")
plt.tight_layout()
plt.savefig("distribuicao_risco.png")
plt.close()

# 2. Chuva vs Nível do Rio
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, x="chuva_mm", y="nivel_metro", hue="nivel_categoria")
plt.title("Chuva vs Nível do Rio")
plt.xlabel("Chuva (mm)")
plt.ylabel("Nível do Rio (m)")
plt.tight_layout()
plt.savefig("chuva_vs_nivel_categoria.png")
plt.close()

# 3. Média de Vazão por Categoria
plt.figure(figsize=(6, 4))
df.groupby("nivel_categoria")["vazao_m3s"].mean().plot(kind="bar")
plt.title("Média de Vazão por Categoria de Risco")
plt.ylabel("Vazão Média (m³/s)")
plt.tight_layout()
plt.savefig("media_vazao_por_categoria.png")
plt.close()


ModuleNotFoundError: No module named 'matplotlib'